In [1]:
DISCRIMINATOR_LR = 0.00005
DISCRIMINATOR_WD = 1e-5
POLICY_LR = 0.0001
LAMBDA_1 = 0.1
LAMBDA_2 = 0.12
EPOCH = 50
BATCH_SIZE = 8
TASK = 'coffee'
VARIANCE = '1'    # D0 or D1

## Setup

In [2]:
import gym
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from torch.autograd import Variable
from torch.distributions import Categorical, Normal

In [3]:
import os
# First, we need to decide where to host the runtime storage
USE_GDRIVE_STORAGE = True

if not USE_GDRIVE_STORAGE:
    # Option 1: use the colab runtime storage. All trained model and downloaded
    # will disappear after you disconnect from the runtime.
    WS_DIR = "/content/"
else:
    # Option 2: use your google drive as the runtime storage. You need to grant
    # permission for the colab runtime to access your google drive. You also
    # need to decide on a workspace for robomimic. In this case, we've created a
    # folder called "colab_ws" in Google Drive.
    from google.colab import drive
    drive.mount('/content/drive')
    WS_DIR = "/content/drive/MyDrive/colab_ws/" # this should be the absolute path, e.g., "/content/drive/MyDrive/my-ws/"
    assert os.path.exists(WS_DIR)

%cd $WS_DIR

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/colab_ws


In [4]:
# Install the basic requirements
%cd $WS_DIR
!pip install -e robosuite/
!pip install -e robomimic/
!pip install -e mimicgen_environments/
!pip install mujoco

import sys
import os
sys.path.append('./robosuite/')
sys.path.append('./robomimic/')
sys.path.append('./mimicgen_environments/')

/content/drive/MyDrive/colab_ws
Obtaining file:///content/drive/MyDrive/colab_ws/robosuite
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for robosuite (pyproject.toml) ... done
  Created wheel for robosuite: filename=robosuite-1.4.1-0.editable-py3-none-any.whl size=6804 sha256=ef26163a4ecd25e29cb6e66b3e9ba0b0dfb7c971aff87f133e81ea00e06eda33
  Stored in directory: /tmp/pip-ephem-wheel-cache-rd74c4qn/wheels/07/97/34/c35a2d0b7f1dab8a60b73d241ad0302d7d627ab89e6a23c480
Successfully built robosuite
  Attempting uninstall: robosuite
    Found existing installation: robosuite 1.4.1
    Uninstalling robosuite-1.4.1:
      Successfully uninstalled robosuite-1.4.1
Obtaining file:///content/drive/MyDrive/colab_ws/robomimic
  Preparing metadata (setup.py) ... done
  Attempting uninstall: robomimic
    Found existing ins

In [5]:
import json
import h5py
import numpy as np

# enforce that the dataset exists
DATA_DIR = WS_DIR + "mimicgen_data/"
dataset_path = os.path.join(DATA_DIR, TASK + "_d" + VARIANCE + "_100.hdf5")
assert os.path.exists(dataset_path)

## Architectures

In [6]:
class Policy(nn.Module):
    def __init__(self, img_dim, combined_dim, action_dim):
        super(Policy, self).__init__()
        self.im = nn.Sequential(
            nn.Conv2d(img_dim[2], 32, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Flatten()
        )

        self.feature_dim = 128 * (img_dim[0] // 8 + 1) * (img_dim[1] // 8 + 1)

        self.fully_connected = nn.Sequential(
            nn.Linear(self.feature_dim + combined_dim, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 64)
        )

        self.action_mean = nn.Linear(64, action_dim)
        self.action_mean.weight.data.mul_(0.1)
        self.action_mean.bias.data.mul_(0.1)

        self.action_log_std = nn.Parameter(abs(torch.randn(1, action_dim)))

    def forward(self, img, combined_features):
        x = self.im(img)
        x = torch.cat([x, combined_features], dim=-1)
        x = self.fully_connected(x)

        action_mean = self.action_mean(x)

        return action_mean

    def log_std_and_std(self, action_means):
        action_log_std = self.action_log_std.expand_as(action_means)
        return action_log_std, torch.exp(action_log_std)

In [7]:
class Discriminator(nn.Module):
    def __init__(self, img_dim, combined_dim, action_dim):
        super(Discriminator, self).__init__()
        self.im = nn.Sequential(
            nn.Conv2d(img_dim[2], 32, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Flatten()
        )

        self.feature_dim = 128 * (img_dim[0] // 8 + 1) * (img_dim[1] // 8 + 1)

        self.fully_connected = nn.Sequential(
            nn.Linear(self.feature_dim + combined_dim + action_dim, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, img, combined_features, action):
        x = self.im(img)
        x = torch.cat([x, combined_features, action], dim=-1)
        return self.fully_connected(x)

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader

class MimicGenDataset(Dataset):
    def __init__(self, file_path):
        self.file = h5py.File(file_path, 'r')
        self.demos = list(self.file['data'].keys())
        inds = np.argsort([int(elem.split('_')[1]) for elem in self.demos])
        self.demos = [self.demos[i] for i in inds]
        random.shuffle(self.demos)
        self.combined_dim = None

    def __len__(self):
        return len(self.demos)

    def __getitem__(self, idx):
        demo_key = self.demos[idx]
        demo_grp = self.file['data'][demo_key]

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        e_imgs = torch.tensor(demo_grp['obs']['agentview_image'][:], dtype=torch.float32).permute(0, 3, 1, 2).to(device)
        e_obj = torch.tensor(demo_grp['obs']['object'][:], dtype=torch.float32).to(device)
        e_eef_pos = torch.tensor(demo_grp['obs']['robot0_eef_pos'][:], dtype=torch.float32).to(device)
        e_eef_quat = torch.tensor(demo_grp['obs']['robot0_eef_quat'][:], dtype=torch.float32).to(device)
        e_joint_pos = torch.tensor(demo_grp['obs']['robot0_joint_pos'][:], dtype=torch.float32).to(device)
        e_actions = torch.tensor(demo_grp['actions'][:], dtype=torch.float32).to(device)
        e_states = torch.tensor(demo_grp['states'][:], dtype=torch.float32).to(device)

        combined_features = torch.cat([e_obj, e_eef_pos, e_eef_quat, e_joint_pos, e_states], dim=1)

        return e_imgs, combined_features, e_actions

    def close(self):
        self.file.close()

## Training Setup

In [9]:
import json
import h5py
import numpy as np

# enforce that the dataset exists
DATA_DIR = WS_DIR + "mimicgen_data/"
dataset_path = os.path.join(DATA_DIR, TASK + "_d" + VARIANCE + "_100.hdf5")
assert os.path.exists(dataset_path)

In [10]:
dataset = MimicGenDataset(dataset_path)
data_loader = DataLoader(
    dataset=dataset,
    sampler=None,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [11]:
# def train_info_gail(policy, discriminator, lambda1, lambda2, expert, demo):
def train_info_gail(policy, discriminator, lambda1, lambda2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    discriminator.to(device)
    policy.to(device)

    discriminator_optimizer = optim.SGD(discriminator.parameters(), lr=DISCRIMINATOR_LR, weight_decay=DISCRIMINATOR_WD)
    policy_optimizer = optim.Adam(policy.parameters(), lr=POLICY_LR)

    adv_loss = nn.BCELoss()
    mse_loss = nn.MSELoss()

    for epoch in range(EPOCH):
        # random.shuffle(demos)

        i = 0

        progress_bar = tqdm(enumerate(dataset), total=len(dataset), desc=f"Epoch {epoch}")
        # progress_bar = tqdm(data_loader, desc=f"Epoch {epoch}")
        # for i, demo_key in progress_bar:
        for i, data in progress_bar:

            e_imgs, combined_features, e_actions = data

            real_labels = torch.ones(e_actions.shape[0], 1).to(device)
            fake_labels = torch.zeros(e_actions.shape[0], 1).to(device)

            # Update Discriminator
            discriminator_optimizer.zero_grad()
            real_predictions = discriminator(e_imgs, combined_features, e_actions)
            fake_actions = policy(e_imgs, combined_features).detach()
            fake_predictions = discriminator(e_imgs, combined_features, fake_actions)
            discriminator_loss = adv_loss(real_predictions, real_labels) + adv_loss(fake_predictions, fake_labels)
            discriminator_loss.backward()
            discriminator_optimizer.step()

            # Update Generator
            policy_optimizer.zero_grad()
            fake_actions = policy(e_imgs, combined_features)
            fake_predictions = discriminator(e_imgs, combined_features, fake_actions)
            generator_loss = lambda1 * adv_loss(fake_predictions, real_labels)
            bc_loss = lambda2 * mse_loss(fake_actions, e_actions)
            total_policy_loss = generator_loss + bc_loss
            total_policy_loss.backward()
            policy_optimizer.step()

            progress_bar.set_description(f"""Epoch {epoch+1} | Iter {i} | Discriminator Loss: {discriminator_loss.item():.4f} | Policy Loss: {total_policy_loss.item():.4f}""")
            # i += 1

In [12]:
import robomimic.utils.obs_utils as ObsUtils

# We normally need to make sure robomimic knows which observations are images (for the
# data processing pipeline). This is usually inferred from your training config, but
# since we are just playing back demonstrations, we just need to initialize robomimic
# with a dummy spec.
dummy_spec = dict(
    obs=dict(
            low_dim=["robot0_eef_pos"],
            rgb=[],
        ),
)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=dummy_spec)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: []


In [13]:
import robomimic.utils.env_utils as EnvUtils

f = h5py.File(dataset_path, "r")
env_meta = json.loads(f["data"].attrs["env_args"])

# create simulation environment from environment metedata
env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta,
    render=False,            # no on-screen rendering
    render_offscreen=True,   # off-screen rendering to support rendering video frames
)

/content/drive/MyDrive/colab_ws/./robosuite/robosuite/macros.py:53: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  ROBOSUITE_DEFAULT_LOGGER.warn("No private macro file found!")
[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /content/drive/MyDrive/colab_ws/./robosuite/robosuite/scripts/setup_macros.py (macros.py:55)


Created environment with name Coffee_D1
Action size is 7


In [14]:
_, combined_features, _ = dataset[0]
combined_dim = combined_features.shape[1]
print(combined_dim)

118


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Training

In [15]:
policy = Policy((84, 84, 3), combined_dim, 7)
discriminator = Discriminator((84, 84, 3), combined_dim, 7)
lambda1 = LAMBDA_1
lambda2 = LAMBDA_2

train_info_gail(policy, discriminator, lambda1, lambda2)

string = WS_DIR + TASK + '_policy.pt'
torch.save(policy, string)

Epoch 1 | Iter 99 | Discriminator Loss: 1.4036 | Policy Loss: 0.0980: 100%|██████████| 100/100 [00:09<00:00, 10.15it/s]
Epoch 2 | Iter 99 | Discriminator Loss: 1.3993 | Policy Loss: 0.0958: 100%|██████████| 100/100 [00:08<00:00, 11.44it/s]
Epoch 3 | Iter 99 | Discriminator Loss: 1.3962 | Policy Loss: 0.0900: 100%|██████████| 100/100 [00:08<00:00, 11.74it/s]
Epoch 4 | Iter 99 | Discriminator Loss: 1.3949 | Policy Loss: 0.0893: 100%|██████████| 100/100 [00:08<00:00, 11.76it/s]
Epoch 5 | Iter 99 | Discriminator Loss: 1.3939 | Policy Loss: 0.0875: 100%|██████████| 100/100 [00:08<00:00, 11.59it/s]
Epoch 6 | Iter 99 | Discriminator Loss: 1.3929 | Policy Loss: 0.0847: 100%|██████████| 100/100 [00:08<00:00, 11.67it/s]
Epoch 7 | Iter 99 | Discriminator Loss: 1.3923 | Policy Loss: 0.0825: 100%|██████████| 100/100 [00:08<00:00, 11.72it/s]
Epoch 8 | Iter 99 | Discriminator Loss: 1.3916 | Policy Loss: 0.0831: 100%|██████████| 100/100 [00:08<00:00, 11.75it/s]
Epoch 9 | Iter 99 | Discriminator Loss: 

## Rollouut

In [16]:
import imageio

# prepare to write playback trajectories to video
video_path = os.path.join(DATA_DIR, TASK + "_playback.mp4")
video_writer = imageio.get_writer(video_path, fps=20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
def playback_trajectory(demo_key):
    """
    Simple helper function to playback the trajectory stored under the hdf5 group @demo_key and
    write frames rendered from the simulation to the active @video_writer.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # robosuite datasets store the ground-truth simulator states under the "states" key.
    # We will use the first one, alone with the model xml, to reset the environment to
    # the initial configuration before playing back actions.
    init_state = f["data/{}/states".format(demo_key)][0]
    model_xml = f["data/{}".format(demo_key)].attrs["model_file"]
    initial_state_dict = dict(states=init_state, model=model_xml)

    # reset to initial state
    env.reset_to(initial_state_dict)

    # playback actions one by one, and render frames

    e_actions = f["data/{}/actions".format(demo_key)][:]

    # Generate trajectories
    state = env.get_state()['states']
    ob = env.get_observation()

    img = env.render(mode="rgb_array", height=84, width=84, camera_name="agentview")
    obj = ob['object']

    eef_pos = ob['robot0_eef_pos']
    eef_quat = ob['robot0_eef_quat']
    joint_pos = ob['robot0_joint_pos']


    for t in tqdm(range(1000)):
        img_tensor = torch.tensor(img.copy(), dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2).to(device)

        obj_tensor = torch.tensor(obj, dtype=torch.float32).unsqueeze(0).to(device)
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        eef_pos_tensor = torch.tensor(eef_pos, dtype=torch.float32).unsqueeze(0).to(device)
        eef_quat_tensor = torch.tensor(eef_quat, dtype=torch.float32).unsqueeze(0).to(device)
        joint_pos_tensor = torch.tensor(joint_pos, dtype=torch.float32).unsqueeze(0).to(device)


        combined_features = torch.cat([obj_tensor, eef_pos_tensor, eef_quat_tensor, joint_pos_tensor, state_tensor], dim=1)


        action_mean = policy(img_tensor, combined_features)
        action_log_std, action_std = policy.log_std_and_std(action_mean)
        action = action_mean
        env.step(action.detach().cpu().numpy()[0])
        state = env.get_state()['states']
        ob = env.get_observation()
        img = env.render(mode="rgb_array", height=84, width=84, camera_name="agentview")
        obj = ob['object']
        video_img = env.render(mode="rgb_array", height=512, width=512, camera_name="agentview")
        video_writer.append_data(video_img)

In [18]:
# playback the first 3 demos and record them to a video file
for ep in dataset.demos[:1]:
    print("Playing back demo key: {}".format(ep))
    playback_trajectory(ep)

# done writing video
video_writer.close()

Playing back demo key: demo_540


  0%|          | 0/1000 [00:00<?, ?it/s]/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
100%|██████████| 1000/1000 [03:59<00:00,  4.17it/s]


In [ ]:
# view the trajectories!
from IPython.display import Video
Video(video_path, embed=True)

## Evaluation

In [ ]:
eval_path = os.path.join(DATA_DIR, TASK + "_d" + VARIANCE + "_test.hdf5")
testset = MimicGenDataset(eval_path)
test_loader = DataLoader(
    dataset=testset,
    sampler=None,
    batch_size=8,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [ ]:
_, combined_features, _ = testset[0]
combined_dim = combined_features.shape[1]

In [22]:
model = Policy((84, 84, 3), combined_dim, 7)
save_position = WS_DIR + TASK + '_policy.pt'
model = torch.load(save_position)
model.eval()
test_g_loss = 0.0
num_batches = 0

# progress_bar = tqdm(test_loader, desc=f'Eval')
progress_bar = tqdm(enumerate(testset), total=len(testset), desc=f"Evaluation")

for i, data in progress_bar:
# for data in progress_bar:
    e_imgs, combined_features, e_actions = data

    g_outputs = model(e_imgs, combined_features)
    g_loss = nn.MSELoss()(g_outputs, e_actions)
    test_g_loss += g_loss.item()

    num_batches += 1

    progress_bar.set_description(f"Average MSE Loss: {test_g_loss/num_batches:.4f}")

Average MSE Loss: 0.1167: 100%|██████████| 100/100 [00:05<00:00, 16.87it/s]
